In [4]:
# from langchain.llms.openai import OpenAI
# from langchain.chat_models import ChatOpenAI

# # you can pass OPENAI_API_KEY as arguments for the methods below, but recommend .env option
# llm = OpenAI(model_name="gpt-3.5-turbo-instruct")
# chat = ChatOpenAI(model_name="gpt-3.5-turbo") # I can easily use antrophic

# a = llm.predict("How many planets are there?")
# b = chat.predict("How many planets are there?")

In [6]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    temperature=0.1
)

'\n\nAs of April 2021, there are 8 planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. However, there may be more planets beyond our solar system that have yet to be discovered.'

In [12]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}?")

prompt = template.format(country_a="Mexico", country_b="Thailand")

chat.predict(prompt)

# messages = [
#     SystemMessage(
#         content="You are a geography expert. And you only reply in {language}."
#     ),
#     AIMessage(content="My name is {name}"),
#     HumanMessage(content="What is the distance between {country_a} and {country_b}. Also, tell me what is your name?")
# ]

# chat.predict_messages(messages)

'The distance between Mexico and Thailand is approximately 16,500 kilometers (10,250 miles) when measured in a straight line.'

In [18]:
# LangChain expression language
template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}"),
    ("ai", "My name is {name}"),
    ("human", "What is the distance between {country_a} and {country_b}. Also, tell me what is your name?")
])

prompt = template.format_messages(
    language="Korean",
    name="Jimmy Butler",
    country_a="Korea",
    country_b="Japan"
)

chat.predict_messages(prompt)

AIMessage(content='한국과 일본 사이의 거리는 약 900km입니다. 제 이름은 제이미 버틀러입니다.')

In [20]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):

    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
p = CommaOutputParser()

p.parse("Hello, How, are, you")

['Hello', 'How', 'are', 'you']

In [25]:
template = ChatPromptTemplate.from_messages([

    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma-separated list of max {max_items} in lowercase. DO NOT reply with anything else."),
    ("human", "{question}"),
]
    
)

prompt = template.format_messages(
    max_items=10,
    question="What are the colors?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)

['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [26]:
chain = template | chat | CommaOutputParser()
# chain_two = template_2 | chat | CommaOutputParser()

# all = chain_one | chain_two | output 
chain.invoke({
    "max_items": 5,
    "question": "What are the pokemons?"
})

# prompt template / chat model. / commaoutput parser are needed
# internally, langchain will call format_message, chat.predict, parse for you

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']